Let's read the read all tables present in the html website

In [ ]:
import pandas as pd

tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)

We keep just the first table since it contains the information required.

In [91]:
data=pd.DataFrame(tables[0])

The function below is designed to clean the data by removing the 'not assigned' borought, and convert the variable 'Neigborbooh' to the format required in this assignement !

In [92]:
def clean_data(data):  
    data=data[data['Borough']!='Not assigned'].reset_index(drop=True)
    var=" /"
    res=[]
    for n in data['Neighborhood']:
        if var in n:
            res.append(n.replace(var, ","))
        else:
            res.append(n)
    data['Neighborhood']=res
    data.rename(columns={"Postal code": "PostalCode"},inplace=True)
    return data

The function below is dedicated to get the cordinates from the cvs file **"Geospatial_Coordinates.csv"**, firt i sort the data to be similar the order of the csv file, and then i aggregate the two datasets into one dataframe.

In [93]:
def get_cordinates(data):
    cord=pd.read_csv('Geospatial_Coordinates.csv')
    data['num']=data['PostalCode'].str[1:2]
    data['alpha']=data['PostalCode'].str[2:3]
    data.sort_values(['num','alpha'],ascending=True,inplace=True)
    data.drop(columns=['num','alpha'],inplace=True)
    data['Latitude']=cord['Latitude']
    data['Longitude']=cord['Longitude']
    data.reset_index(drop=True,inplace=True)
    return data

In [94]:
data=clean_data(data)
data=get_cordinates(data)

In [95]:
data.head(30)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.794200,-79.262029
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.778517,-79.346556
3,M1G,Scarborough,Woburn,43.770120,-79.408493
4,M1H,Scarborough,Cedarbrae,43.745906,-79.352188
5,M1J,Scarborough,Scarborough Village,43.728496,-79.495697
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.709060,-79.363452
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.728020,-79.388790
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.667967,-79.367675
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.650571,-79.384568


In [121]:
toronto_data = data[data['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.786947,-79.385975
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.704324,-79.388790
3,M4M,East Toronto,Studio District,43.657162,-79.378937
4,M4N,Central Toronto,Lawrence Park,43.648198,-79.379817


In [ ]:
pip install geopy

In [102]:
from geopy.geocoders import Nominatim
address = 'Toronto, TR'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6561136, -79.392321.


In [108]:
import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [117]:
CLIENT_ID = 'AFXA0O3MLX2DBGBERX40HI5CDHHTIOBNSN0JBIPA3KJ2WG2V' # your Foursquare ID
CLIENT_SECRET = '432C250VYVRE2VCRE4MPIBUD1SEPZPUTILQTL34JAIOFVBP1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AFXA0O3MLX2DBGBERX40HI5CDHHTIOBNSN0JBIPA3KJ2WG2V
CLIENT_SECRET:432C250VYVRE2VCRE4MPIBUD1SEPZPUTILQTL34JAIOFVBP1


In [118]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [122]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [123]:
import requests
LIMIT = 100
radius=500
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Ron

In [151]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [152]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Brockton, Parkdale Village, Exhibition Place",0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.025000,0.0000,0.00
1,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00
5,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.333333,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.046154,0.00,0.0,0.046154,0.015385,0.0000,0.00
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00


In [153]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Brockton, Parkdale Village, Exhibition Place",0.025000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.025000,0.0000,0.00
1,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00
3,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00
5,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00
6,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.333333,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.046154,0.00,0.0,0.046154,0.015385,0.0000,0.00
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.0000,0.00


In [154]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2              Brewery  0.05
3            Gastropub  0.05
4  American Restaurant  0.05


----Business reply mail Processing CentrE----
               venue  freq
0        Pizza Place  0.25
1  Mobile Phone Shop  0.25
2     Sandwich Place  0.25
3           Bus Line  0.25
4           Pharmacy  0.00


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0       Yoga Studio  0.06
1     Garden Center  0.06
2       Pizza Place  0.06
3        Comic Shop  0.06
4  Recording Studio  0.06


----Central Bay Street----
         venue  freq
0     Pharmacy   0.2
1  Coffee Shop   0.2
2         Bank   0.2
3      Butcher   0.2
4  Pizza Place   0.2


----Christie----
                             venue  freq
0                             Park   0.5
1                Food & Drink Shop

In [155]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [156]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Convenience Store,Sandwich Place,Cheese Shop,Pet Store
1,Business reply mail Processing CentrE,Bus Line,Pizza Place,Sandwich Place,Mobile Phone Shop,Women's Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store
2,"CN Tower, King and Spadina, Railway Lands, Har...",Yoga Studio,Smoke Shop,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Pizza Place,Burrito Place
3,Central Bay Street,Coffee Shop,Pizza Place,Bank,Butcher,Pharmacy,Dessert Shop,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
4,Christie,Park,Food & Drink Shop,Falafel Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Doner Restaurant,Dog Run,Discount Store


In [157]:
from sklearn.cluster import KMeans
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 4, 1, 1, 1, 1, 1])

In [164]:
# add clustering labels
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(inplace=True)
toronto_merged.head(30) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.786947,-79.385975,1.0,Japanese Restaurant,Chinese Restaurant,Bank,Café,Women's Store,Diner,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1.0,Greek Restaurant,Coffee Shop,Italian Restaurant,Restaurant,Ice Cream Shop,Bookstore,Furniture / Home Store,Grocery Store,Frozen Yogurt Shop,Brewery
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.704324,-79.388790,1.0,Pizza Place,Dessert Shop,Sandwich Place,Sushi Restaurant,Coffee Shop,Italian Restaurant,Gym,Café,Pharmacy,Restaurant
3,M4M,East Toronto,Studio District,43.657162,-79.378937,1.0,Coffee Shop,Clothing Store,Café,Italian Restaurant,Bubble Tea Shop,Middle Eastern Restaurant,Cosmetics Shop,Japanese Restaurant,Lingerie Store,Electronics Store
4,M4N,Central Toronto,Lawrence Park,43.648198,-79.379817,1.0,Coffee Shop,Restaurant,Café,Hotel,American Restaurant,Gym,Deli / Bodega,Gastropub,Italian Restaurant,Japanese Restaurant
5,M4P,Central Toronto,Davisville North,43.653206,-79.400049,1.0,Café,Coffee Shop,Vietnamese Restaurant,Bar,Vegetarian / Vegan Restaurant,Mexican Restaurant,Gaming Cafe,Dumpling Restaurant,Burger Joint,Chinese Restaurant
6,M4R,Central Toronto,North Toronto West,43.693781,-79.428191,1.0,Field,Trail,Hockey Arena,Tennis Court,Women's Store,Department Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
7,M4S,Central Toronto,Davisville,43.713756,-79.490074,1.0,Park,Construction & Landscaping,Massage Studio,Bakery,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.648960,-79.456325,1.0,Gift Shop,Dessert Shop,Dog Run,Breakfast Spot,Eastern European Restaurant,Italian Restaurant,Bar,Bank,Restaurant,Bookstore
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.636966,-79.615819,1.0,Hotel,Coffee Shop,Intersection,Gym,Sandwich Place,American Restaurant,Mediterranean Restaurant,Burrito Place,Fried Chicken Joint,Middle Eastern Restaurant


In [167]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=5)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters